In [1]:
import json, os
from coordTransform_utils import bd09_to_wgs84

# 转坐标：bd09 -> wgs84

In [2]:
def geojson_coord_convert_bd09_to_wgs84(source_file, save_file, name_append=None):
    """
    source_file：原坐标文件读取路径（完整路径）
    save_file: 转坐标后文件保存路径（完整路径）
    name_append：文件名附加后缀，默认None
    """
    data = json.load(open(source_file, 'r', encoding='utf-8'))
    data['crs']['properties']['name'] = 'urn:ogc:def:crs:OGC:1.3:CRS84'
    features = data['features']
    # starting convert coords
    for idx, feature in enumerate(features):
        ftype = feature['geometry']['type']
        coords = feature['geometry']['coordinates']
        if ftype == 'MultiPoint':
            new_coords = [bd09_to_wgs84(coords[0][0], coords[0][1])]
        elif ftype == 'MultiPolygon':
            new_coords = []
            for mpolygon in coords:
                new_mpolygon = []
                for polygon in mpolygon:
                    new_polygon = []
                    for this_coord in polygon:
                        this_new_coord = bd09_to_wgs84(this_coord[0], this_coord[1])
                        new_polygon.append(this_new_coord)
                    new_mpolygon.append(new_polygon)
                new_coords.append(new_mpolygon)
        elif ftype == 'MultiLineString':
            new_coords = []
            for line in coords:
                new_line = []
                for this_coord in line:
                    this_new_coord = bd09_to_wgs84(this_coord[0], this_coord[1])
                    new_line.append(this_new_coord)
                new_coords.append(new_line)
        elif ftype == 'Point':
            new_coords = bd09_to_wgs84(coords[0], coords[1])
        else:
            print('Invalid geometry type: {}, converting failed'.format(ftype))
            exit()
        features[idx]['geometry']['coordinates'] = new_coords
    data['features'] = features
    if name_append is not None:
        data['name'] = data['name'] + '_' + name_append
    if not os.path.exists(os.path.dirname(save_file)):
        os.makedirs(os.path.dirname(save_file))
    json.dump(data, open(save_file, 'w', encoding='utf-8'), indent=4, ensure_ascii=False)
    print('Coords converting finished, new file saved to: \n{}'.format(os.path.abspath(save_file)))

# MongoDB下载的Json转存为GeoJson

In [3]:
def mongodb_json_to_geojson(folder, src_file, save_file):
    """
    folder: json文件的文件夹
    src_file：json文件的文件名（仅文件名，非完整路径）
    save_file：geojson文件的保存文件名（仅文件名，非完整路径）
    """
    data = json.load(open(os.path.join(folder, src_file), encoding='utf-8'))
    out = {
        "type": "FeatureCollection",
        "name": src_file.split('.')[0],
        "crs": {
            "type": "name",
            "properties": {
                "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
            }
        },
        "features": []
    }
    features = []
    exclude = ['_id', 'block_info']
    for d in data:
        obj = {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Point",
                "coordinates": None
            }
        }
        for k,v  in d.items():
            if k not in exclude:
                obj['properties'][k] = v
        try:
            lng, lat = float(d['coord_lng']), float(d['coord_lat'])
        except Exception as e:
            print(f'\nError: {e}\nRaw data:\n{d}\nRaw data ignored, possibly due to None lng/lat')
            continue
        obj['geometry']['coordinates'] = [lng, lat]
        features.append(obj)
    out['features'] = features
    json.dump(out, open(os.path.join(folder, save_file),'w',encoding='utf-8'),
             indent=4, ensure_ascii=False)

# 例子

## 转坐标

In [4]:
folder = r'D:\L3\to_jiwen'
json_files = [x for x in os.listdir(folder) if x.endswith('.geojson')]
print(json_files)
for src_file in json_files:
    src_path = os.path.join(folder, src_file)
    save_path = os.path.join(folder, 'wgs84', src_file)
    geojson_coord_convert_bd09_to_wgs84(src_path, save_path)

['house_detailed_second_hand_lianjia_futian.geojson', 'block_lianjia_futian.geojson']
Coords converting finished, new file saved to: 
D:\L3\to_jiwen\wgs84\house_detailed_second_hand_lianjia_futian.geojson
Coords converting finished, new file saved to: 
D:\L3\to_jiwen\wgs84\block_lianjia_futian.geojson


## Json to Geojson

In [5]:
folder = r'D:\L3\to_jiwen'
json_files = [x for x in os.listdir(folder) if x.endswith('.json')]
print(json_files)
for src_file in json_files:
    save_file = f"{src_file.split('.')[0]}.geojson"
    mongodb_json_to_geojson(folder, src_file, save_file)

['house_detailed_second_hand_lianjia_futian.json', 'block_lianjia_futian.json']

Error: float() argument must be a string or a number, not 'NoneType'
Raw data:
{'_id': {'$oid': '6121deddee47a81518fdd189'}, 'url': 'https://sz.lianjia.com/ershoufang/105108223069.html', 'title': '2/9号线景田站，红本在手，满五唯一，业主诚售', 'summary_info': '1室1厅 | 41.35平米 | 南 | 简装 | 高楼层(共30层) | 2003年建 | 塔楼', 'summary_total_price': '313', 'summary_unit_price': '单价75500元/平米', 'price': '313万', 'unit_price': '75500元/平米', 'layout': '1室1厅1厨1卫', 'floor': '高楼层 (共30层)', 'fa': '41.35㎡', 'v_struct': '平层', 'area_in': '35.55㎡', 'bldg_type': '塔楼', 'orient': '南', 'bldg_struct': '钢混结构', 'decorat': '简装', 'ele_ratio': '三梯十四户', 'ele': '有', 'market_time': '2021-08-03', 'last_trade_time': '2005-12-22', 'years': '满五年', 'ass_id': '210622393649', 'ownership_type': '商品房', 'usage_type': '普通住宅', 'property_right': '非共有', 'layout_detailed': [{'name': '客厅', 'area': '12.89平米', 'orient': '西南', 'window': '普通窗'}, {'name': '卧室', 'area': '7.48平米', 'orient': '